In [10]:
import sys
#!{sys.executable} -m pip install fiona
#!{sys.executable} -m pip install rasterio
#!{sys.executable} -m pip install shapely
#!{sys.executable} -m pip install geopandas
!{sys.executable} -m pip install tqdm
!{sys.executable} -m !pip install time


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://imaxt.ast.cam.ac.uk/pip/
/opt/conda/bin/python: No module named !pip


In [19]:

import os

import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union

import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

from scipy import ndimage
from tqdm import tqdm
import time

#https://lpsmlgeo.github.io/2019-09-22-binary_mask/
def generate_mask(raster_path, shape_path, file_name):
    
    """Function that generates a binary mask from a vector file (shp or geojson)
    
    raster_path = path to the .tif;

    shape_path = path to the shapefile or GeoJson.

    output_path = Path to save the binary mask.

    file_name = Name of the file.
    
    """
    
    #load raster
    
    with rasterio.open(raster_path, "r") as src:
        raster_img = src.read()
        raster_meta = src.meta
    
    #load o shapefile ou GeoJson
    train_df = gpd.read_file(shape_path)
    
    #Verify crs
    if train_df.crs != src.crs:
        print(" Raster crs : {}, Vector crs : {}.\n Convert vector and raster to the same CRS.".format(src.crs,train_df.crs))
        
        
    #Function that generates the mask
    def poly_from_utm(polygon, transform):
        poly_pts = []

        poly = cascaded_union(polygon)
        for i in np.array(poly.exterior.coords):
            poly_pts.append(~transform * tuple(i))

        new_poly = Polygon(poly_pts)
        return new_poly
    
    mask_list = []
    count = 1
    im_size = (src.meta['height'], src.meta['width'])
    print(im_size)
    
    mask_all = np.zeros(im_size).astype(bool)
    
    for num, row in tqdm(train_df.iterrows()):
        poly_shp = []
        if row['geometry'].geom_type == 'Polygon':
            poly = poly_from_utm(row['geometry'], src.meta['transform'])
            poly_shp.append(poly)
        else:
            for p in row['geometry']:
                poly = poly_from_utm(p, src.meta['transform'])
                poly_shp.append(poly)
        new_mask = rasterize(shapes=poly_shp, out_shape=im_size)
        #mask = ndimage.distance_transform_edt(new_mask)
        #mask_list.append(mask)
        mask_all = mask_all + new_mask.astype(bool)
    
    #masks = np.asarray(mask_list)
    #mask = np.argmax(masks, axis=0)
    
    #Salve
    mask_all = mask_all.astype(bool)
    im = Image.fromarray(mask_all)
    im.save(file_name)
    
    # bin_mask_meta = src.meta.copy()
    # bin_mask_meta.update({'count': 1})
    # #os.chdir(output_path)
    # with rasterio.open(file_name, 'w', **bin_mask_meta) as dst:
    #     dst.write(mask, 1)

    # plt.rcParams["figure.figsize"] = (50,50)
    # fig = plt.figure()
    # ax1 = fig.add_subplot(5,5,1)
    # ax1.imshow(mask)


In [ ]:
# generate_mask("17.11.20_HCC38_1_7168_9600.tif", "17.11.20_HCC38_1_7168_9600.geojson", "output_path", "file_name.tif")
# generate_mask("Glioma_5_s1z1c5_ORG.tif", "Glioma_5_s1z1c5_ORG.geojson", "output_path", "file_name.tif")
# generate_mask("FullInferenceGrahamProcessed/790063_vessel.png", "FullInferenceGrahamProcessed/790063_vessel.geojson", "output_path", "file_name.png")

import glob


for file in glob.glob("FullInferenceGrahamCleaned/*.geojson"):

    print(file)
    name = os.path.basename(file[:-15])
    print(name)
    
    image_name = "FullInferenceGrahamProcessed/" + name + "vessel.png"
    output = "FullInferenceGrahamCleaned/" + name + "cleaned.png"
    
    generate_mask(image_name, file, output)






FullInferenceGrahamCleaned/790046_cleaned.geojson
790046_


/opt/conda/lib/python3.9/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


 Raster crs : None, Vector crs : epsg:4326.
 Convert vector and raster to the same CRS.
(8252, 31872)


0it [00:00, ?it/s]/tmp/ipykernel_1404/93537061.py:55: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
1395it [10:58,  2.12it/s]


FullInferenceGrahamCleaned/790035_cleaned.geojson
790035_


/opt/conda/lib/python3.9/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


 Raster crs : None, Vector crs : epsg:4326.
 Convert vector and raster to the same CRS.
(34933, 21912)


0it [00:00, ?it/s]/tmp/ipykernel_1404/93537061.py:55: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
159it [03:29,  1.30s/it]